In [1]:
import csv
from pathlib import Path

from ns_vfs.common.utility import save_frames
from ns_vfs.config.loader import load_config
from ns_vfs.data.frame import BenchmarkLTLFrame, FramesofInterest
from ns_vfs.frame_searcher import FrameSearcher
from ns_vfs.model.vision.grounding_dino import GroundingDino
from ns_vfs.processor.benchmark_video_processor import BenchmarkVideoFrameProcessor
from ns_vfs.video_to_automaton import VideotoAutomaton
from _common import get_available_benchmark_video, get_classification_score
from ns_vfs.model.vision.yolo import Yolo 
from ns_vfs.common.utility import list_flatten, load_pickle_to_dict, save_dict_to_pickle

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = load_config()
benchmark_frame_video_root_dir = Path(
    "/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video"
)
benchmark_image_set_dir = [x for x in benchmark_frame_video_root_dir.iterdir() if x.is_dir()]
cv_model_list = ["yolo"]

In [3]:
ltl_video_dir_set = [x for x in benchmark_image_set_dir[0].iterdir() if x.is_dir()]

In [9]:
ltl_video_dir_set

[PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/Gprop2'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/prop1Uprop2'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/(prop1&prop2)Uprop3'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/prop1&prop2'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/Fprop1')]

In [8]:
benchmark_video_file_list = get_available_benchmark_video(ltl_video_dir_set[2])
benchmark_video_file_list

[PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/(prop1&prop2)Uprop3/benchmark_COCO_ltl_("person" & "surfboard") U "spoon"_150_1.pkl'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/(prop1&prop2)Uprop3/benchmark_COCO_ltl_("banana" & "keyboard") U "mouse"_25_1.pkl'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/(prop1&prop2)Uprop3/benchmark_COCO_ltl_("car" & "person") U "tie"_125_4.pkl'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/(prop1&prop2)Uprop3/benchmark_COCO_ltl_("scissors" & "book") U "backpack"_25_4.pkl'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/(prop1&prop2)Uprop3/benchmark_COCO_ltl_("person" & "skis") U "book"_200_0.pkl'),
 PosixPath('/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/_validated_benchmark_video/coco/(prop1&prop2)Uprop3/benc

In [ ]:
"""
Taking a single benchmark video
"""
search_result_per_video = {}
####################################################
benchmark_video_file = benchmark_video_file_list[0]
for cv_model in cv_model_list:
    if cv_model == "yolo":
        cv_detection_model = Yolo(config=config.YOLO,
                                    weight_path=config.YOLO.YOLO_CHECKPOINT_PATH)
    elif cv_model == "grounding_dino":
        cv_detection_model = GroundingDino(
                config=config.GROUNDING_DINO,
                weight_path=config.GROUNDING_DINO.GROUNDING_DINO_CHECKPOINT_PATH,
                config_path=config.GROUNDING_DINO.GROUNDING_DINO_CONFIG_PATH,
            )
    benchmark_video_processor = BenchmarkVideoFrameProcessor(
        video_path=benchmark_video_file,
        artifact_dir=config.VERSION_AND_PATH.ARTIFACTS_PATH)

    benchmark_video: BenchmarkLTLFrame = benchmark_video_processor.benchmark_image_frames

    video_automata_builder = VideotoAutomaton(
        detector=cv_detection_model,
        video_processor=benchmark_video_processor,
        artifact_dir=config.VERSION_AND_PATH.ARTIFACTS_PATH,
        proposition_set=benchmark_video.proposition,
        save_annotation=False,  # TODO: Debug only
        save_image=False,  # TODO: Debug only
        ltl_formula=f"P>=0.80 [{benchmark_video.ltl_formula}]",
        verbose=False,
    )
    frame_sercher = FrameSearcher(
        video_automata_builder=video_automata_builder,
        video_processor=benchmark_video_processor,
    )

    frame_of_interest = frame_sercher.search()

    search_result_per_video["benchmark_video"] = benchmark_video
    search_result_per_video[cv_model] = frame_of_interest

# classification_metrics
search_result_per_video = get_classification_score(search_result_per_video)

In [ ]:
"""
Taking a single benchmark video from ltl_video_dir_set
"""
result = {}
search_result_per_video = {}
####################################################
for benchmark_video_file in benchmark_video_file_list[0:5]:
    ltl_formula = benchmark_video_file.name.split(".")[0].split("_ltl_")[-1]
    #result[ltl_formula] = {}
    for cv_model in cv_model_list:
        if cv_model == "yolo":
            cv_detection_model = Yolo(config=config.YOLO,
                                        weight_path=config.YOLO.YOLO_CHECKPOINT_PATH)
        elif cv_model == "grounding_dino":
            cv_detection_model = GroundingDino(
                    config=config.GROUNDING_DINO,
                    weight_path=config.GROUNDING_DINO.GROUNDING_DINO_CHECKPOINT_PATH,
                    config_path=config.GROUNDING_DINO.GROUNDING_DINO_CONFIG_PATH,
                )
        benchmark_video_processor = BenchmarkVideoFrameProcessor(
            video_path=benchmark_video_file,
            artifact_dir=config.VERSION_AND_PATH.ARTIFACTS_PATH)

        benchmark_video: BenchmarkLTLFrame = benchmark_video_processor.benchmark_image_frames

        video_automata_builder = VideotoAutomaton(
            detector=cv_detection_model,
            video_processor=benchmark_video_processor,
            artifact_dir=config.VERSION_AND_PATH.ARTIFACTS_PATH,
            proposition_set=benchmark_video.proposition,
            save_annotation=False,  # TODO: Debug only
            save_image=False,  # TODO: Debug only
            ltl_formula=f"P>=0.80 [{benchmark_video.ltl_formula}]",
            verbose=False,
        )
        frame_sercher = FrameSearcher(
            video_automata_builder=video_automata_builder,
            video_processor=benchmark_video_processor,
        )

        frame_of_interest = frame_sercher.search()

        search_result_per_video["benchmark_video"] = benchmark_video
        search_result_per_video[cv_model] = frame_of_interest

    # classification_metrics
    search_result_per_video = get_classification_score(search_result_per_video)
    result[ltl_formula] = search_result_per_video

In [ ]:
result.keys()

In [ ]:
save_dict_to_pickle(result, path="/opt/Neuro-Symbolic-Video-Frame-Search/artifacts/", file_name="test_result.pkl")

In [ ]:
result.keys()

In [ ]:
result['F "horse"_25_2'].keys()

In [ ]:
result['F "horse"_25_2']["benchmark_video"]
print(result['F "horse"_25_2']["benchmark_video"].number_of_frame)
print(result['F "horse"_25_2']["benchmark_video"].frames_of_interest)
#print(result['F "horse"_25_2']["benchmark_video"].labels_of_frames)
#print(result['F "horse"_25_2']["benchmark_video"].images_of_frames)

In [ ]:
result['F "horse"_25_2']["grounding_dino"]
print(result['F "horse"_25_2']["grounding_dino"].foi_list)
# print(result['F "horse"_25_2']["grounding_dino"].frame_images)

In [ ]:
result['F "horse"_25_2']["yolo"]
print(result['F "horse"_25_2']["yolo"].foi_list)
# print(result['F "horse"_25_2']["grounding_dino"].frame_images)

In [ ]:
result['F "horse"_25_2']["grounding_dino_classification_metrics"]

In [ ]:
result['F "horse"_25_2']["yolo_classification_metrics"]